In [1]:
import numpy as np
from cities.utils.cleaning_utils import find_repo_root
import pandas as pd
import numpy as np

root = find_repo_root()


In [2]:
import requests

# https://apps.bea.gov/api/signup/ link for registering in order to obtain BEA api key 

api_key = "ED8C2AB7-DD09-4EE0-AC91-116A9E05348A" # private api key for BEA data


years = '1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021'


# real gross domewstic product?

url = f"https://apps.bea.gov/api/data/?UserID={api_key}&method=GetData&datasetname=Regional&TableName=CAGDP1&LineCode=1&Year={years}&GeoFips=MSA&ResultFormat=json"
response = requests.get(url)

assert response.status_code == 200 # 200 means success


data = response.json()
data_series = data['BEAAPI']['Results']['Data']

df = pd.DataFrame(data_series)

In [6]:
gdp_ma = df.copy()

gdp_ma = gdp_ma[gdp_ma['GeoFips'].str.endswith('0')]

gdp_ma = gdp_ma[['GeoFips', 'GeoName', 'TimePeriod', 'DataValue']]
gdp_ma = gdp_ma.rename(columns={'DataValue': 'Value', 'TimePeriod': 'Year', 'GeoFips': 'GeoFIPS' })
gdp_ma = gdp_ma.sort_values(by=['GeoFIPS', 'Year']).reset_index(drop=True)
gdp_ma['Value'] = gdp_ma['Value'].astype(float)
gdp_ma['GeoName'] = gdp_ma['GeoName'].astype(str)
gdp_ma['GeoFIPS'] = gdp_ma['GeoFIPS'].astype(np.int64)
gdp_ma['GeoName'] = gdp_ma['GeoName'].str.extract(r'^(.*?)\s*\(', expand=False).fillna('')
gdp_ma['GeoName'] = gdp_ma['GeoName'] + ' (MA)'

gdp_ma.head()

,GeoFIPS,GeoName,Year,Value
0,10180,"Abilene, TX (MA)",2017,6827152.0
1,10180,"Abilene, TX (MA)",2018,7141273.0
2,10180,"Abilene, TX (MA)",2019,7426609.0
3,10180,"Abilene, TX (MA)",2020,7354374.0
4,10180,"Abilene, TX (MA)",2021,7573774.0


In [7]:
gdp_wide = gdp_ma.pivot(index=['GeoFIPS', 'GeoName'], columns='Year', values='Value')
gdp_wide = gdp_wide.reset_index()
gdp_wide.columns.name = None
gdp_wide.head()

,GeoFIPS,GeoName,2017,2018,2019,2020,2021
0,10180,"Abilene, TX (MA)",6827152.0,7141273.0,7426609.0,7354374.0,7573774.0
1,10420,"Akron, OH (MA)",36424857.0,36883382.0,37471099.0,36099097.0,37510049.0
2,10500,"Albany, GA (MA)",5693673.0,5702355.0,5839867.0,5797644.0,6068572.0
3,10540,"Albany-Lebanon, OR (MA)",4505965.0,4802294.0,4921026.0,4788775.0,4893295.0
4,10580,"Albany-Schenectady-Troy, NY (MA)",61024034.0,61119269.0,64250467.0,62574116.0,66759654.0


In [9]:
gdp_wide.to_csv(f"{root}/data/raw/gdp_ma.csv", index=False)   